<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/main/Class_05_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

**Module 5: Regularization and Dropout**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Integrative Biology](https://sciences.utsa.edu/integrative-biology/), [UTSA](https://www.utsa.edu/)

### Module 5 Material

* Part 5.1: Part 5.1: Introduction to Regularization: Ridge and Lasso
* Part 5.2: Using K-Fold Cross Validation with Keras
* Part 5.3: Using L1 and L2 Regularization with Keras to Decrease Overfitting
* Part 5.4: Drop Out for Keras to Decrease Overfitting
* **Part 5.5: Benchmarking Keras Deep Learning Regularization Techniques**



### Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.

In [ ]:
# YOU MUST RUN THIS CELL FIRST

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Part 5.5: Benchmarking Regularization Techniques

Quite a few hyperparameters have been introduced so far.  Tweaking each of these values can have an effect on the score obtained by your neural networks.  Some of the hyperparameters seen so far include:

* Number of layers in the neural network
* How many neurons in each layer
* What activation functions to use on each layer
* Dropout percent on each layer
* L1 and L2 values on each layer

To try out each of these hyperparameters you will need to run train neural networks with multiple settings for each hyperparameter.  However, you may have noticed that neural networks often produce somewhat different results when trained multiple times.  This is because the neural networks start with random weights.  Because of this it is necessary to fit and evaluate a neural network several times to ensure that one set of hyperparameters are actually better than another.  **_Bootstrapping_** can be an effective means of benchmarking (comparing) two sets of hyperparameters.  

Bootstrapping is similar to cross-validation.  Both go through a number of cycles/folds providing validation and training sets.  However, bootstrapping can have an unlimited number of cycles.  Bootstrapping chooses a new train and validation split each cycle, with **_replacement_**.  The fact that each cycle is chosen with replacement means that, unlike cross validation, there will often be repeated rows selected between cycles.  If you run the bootstrap for enough cycles, there will be duplicate cycles.

In this part we will use bootstrapping for **_hyperparameter benchmarking_**.  We will train a neural network for a specified number of splits (denoted by the SPLITS constant).  For these examples we use 100.  We will compare the average score at the end of the 100th epoch.  By the end of the cycles the mean score will have converged somewhat.  This ending score will be a much better basis of comparison than a single cross-validation.  Additionally, the average number of epochs will be tracked to give an idea of a possible optimal value.  Because the early stopping validation set is also used to evaluate the the neural network as well, it might be slightly inflated.  This is because we are both stopping and evaluating on the same sample.  However, we are using the scores only as relative measures to determine the superiority of one set of hyperparameters to another, so this slight inflation should not present too much of a problem.

Because we are benchmarking, we will display the amount of time taken for each cycle.  The following function can be used to nicely format a time span.

### Define functions for this lesson

In [ ]:
import time

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

## Bootstrapping for Regression

Regression bootstrapping uses the **ShuffleSplit** object to perform the splits.  This technique is similar to **KFold** for cross-validation; no balancing occurs.  We will attempt to predict the `age` column for the [Body Performance Dataset](https://www.kaggle.com/datasets/kukuroo3/body-performance-data) that we have used in previous lessons (e.g. Class_05_2, Class_05_3).

## Example 1: Body Performance Dataset

The dataset used for this lesson is the [Body Performance Dataset](https://www.kaggle.com/datasets/kukuroo3/body-performance-data) that we have used in previous lessons (e.g. Class_05_2, Class_05_3).

This dataset has the following 12 categories:

* **age:** 20 ~64
* **gender:** M,F
* **height_cm:** (If you want to convert to feet, divide by 30.48)
* **weight_kg:**
* **body fat_%:**
* **diastolic:** diastolic blood pressure (min)
* **systolic:** systolic blood pressure (min)
* **gripForce:**
* **sit and bend forward_cm:**
* **sit-ups counts:**
* **broad jump_cm:**
* **class:** A,B,C,D ( A: best) / stratified

There are only two columns that are non-numeric (i.e. contain string values): `gender` and `class`.

### Example 1 -Step 1: Create feature vector

The code in the cell below reads the Body Performance dataset, `bodyPerformance.csv` from the course HTTPS server and creates a new DataFrame called `dfBig`. To speedup training, only 15% of `dfBig` is used (about 2,000 subjects) to create the DataFrame,`df`.

In Example 1, the objective is to create a **_regression_** neural network that can predict the `age` of a subject (i.e., the values in the column `age` are the y-values).  

To create our feature vection, we need to convert any string values to numbers. The column `gender` is mapped (`M`=`0`,`F`=`1`) while the column `classes` is One-Hot encoded.

In this lesson only the data in the columns `height_cm`, `weight_kg`, `diastolic`, `systolic` and `gripForce` are standardized.

All of the columns, except `age`, are used for creating the x-value variable. The y-values are generated directly from the values in the column `age`. Both the x-values and the y-values must be converted to type `float32` to avoid errors during training.

As a final check, the `ages` of the first 10 subjects are printed out using the "star" option.

In [ ]:
# Example 1 - Step 1: Create feature vector

import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import zscore

# Read the data set
dfBig = pd.read_csv(
    "https://biologicslab.co/BIO1173/data/bodyPerformance.csv",
    na_values=['NA','?'])

# Only use 15% for neural network
df=dfBig.sample(frac=0.15, random_state=2)

# Map Gender
mapping =  {'M': 0,
            'F': 1}
df['gender'] = df['gender'].map(mapping)

# Generate dummies for class
df = pd.concat([df,pd.get_dummies(df['class'],prefix="class")],axis=1)
df.drop('class', axis=1, inplace=True)

# Standardize ranges
df['height_cm'] = zscore(df['height_cm'])
df['weight_kg'] = zscore(df['weight_kg'])
df['diastolic'] = zscore(df['diastolic'])
df['systolic'] = zscore(df['systolic'])
df['gripForce'] = zscore(df['gripForce'])

# Generate list of columns for x
x_columns = df.columns.drop('age')  # `age` is the y-value

# Generate x-values using x-columns list
x = df[x_columns].values
# Convert x-values to float 32
x = np.asarray(x).astype('float32')

# Generate y-values
y = df['age'].values
# Convert y-values to float 32
y = np.asarray(y).astype('float32')

# Print y categorical names
print(y[0:10])

If your code is correct you should see something similar to the following output:

~~~text
[28. 23. 60. 27. 25. 51. 37. 34. 43. 24.]
~~~

These are the ages of the first 10 subjects in the y-value variable, `y`.

### Example 1 - Step 2: Bootstrapping for Regression

The following code performs the bootstrap. The architecture of the neural network can be adjusted to compare many different configurations. The code below uses a very basic architecture with only 2 hidden layers, and no dropout layers.

The number of times the code "loops" is defined by the variable `SPLITS`. In the example below, `SPLITS=5`. During each loop, the neural network will train for a maximal value specified by the variable `EPOCHS`. In this example, EPOCHS is set to `1000`. However, since the code also implements `EarlyStopping` the number of epochs actually run will usually be less than `100`.

As mentioned above, the uses the `ShuffleSplit()` object to perform the splits for the bootstrapping. This technique is similar to _K_-Fold for cross-validation in the sense that no balancing occurs.

The specific code that generates the `bootstrapping` is provided by the code chunk:

~~~text
# Bootstrap
boot = ShuffleSplit(n_splits=SPLITS, test_size=0.1, random_state=42)
~~~

Keep in mind that bootstrapping chooses a **_new_** train and validation split each cycle, with **_replacement_**.  The fact that each cycle is chosen with replacement means that, unlike cross validation, there will often be **_repeated rows_** selected between cycles.  If you run the bootstrap for enough cycles, there will be duplicate cycles.

In [ ]:
import pandas as pd
import os
import numpy as np
import time
import statistics
from sklearn import metrics
from sklearn.model_selection import ShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.callbacks import EarlyStopping

# Set variables
SPLITS = 5
EPOCHS = 1000

# Record start time
total_start_time = time.time()

# Bootstrap
boot = ShuffleSplit(n_splits=SPLITS, test_size=0.1, random_state=42)

# Track progress
mean_benchmark = []
epochs_needed = []
num = 0

# Consistent tensor shapes
input_shape = x.shape[1]

def create_and_compile_model(input_shape):
    model = Sequential()
    model.add(Input(shape=(input_shape,)))  # Input
    model.add(Dense(20, activation='relu'))  # Hidden 1
    model.add(Dense(10, activation='relu'))  # Hidden 2
    model.add(Dense(1))  # Output only 1 neuron for regression
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Create the model outside the loop
model = create_and_compile_model(input_shape)

print(f"STARTING {SPLITS} BOOTSTRAP CYCLES...")
print("------------------------------------------------------------")

# Loop through samples
for train, test in boot.split(x):
    start_time = time.time()
    num += 1
    print(f"Bootstrap Cycle: {num}")

    # Split train and test
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]

    # Create monitor for early stopping
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
                            patience=15, verbose=0, mode='auto', restore_best_weights=True)

    # Reset model weights before training
    model.set_weights(model.get_weights())

    # Fit model
    history = model.fit(x_train, y_train, validation_data=(x_test, y_test),
                        callbacks=[monitor], verbose=0, epochs=EPOCHS)

    # Record the number of epochs used
    epochs_needed.append(len(history.epoch))

    # Predict on the out-of-boot (validation)
    pred = model.predict(x_test)

    # Measure this bootstrap's log loss
    score = np.sqrt(metrics.mean_squared_error(pred, y_test))
    mean_benchmark.append(score)

    m1 = statistics.mean(mean_benchmark)
    m2 = statistics.mean(epochs_needed)
    print(f"mean_benchmark={mean_benchmark}")

    # Record this iteration
    time_took = time.time() - start_time
    print(f"#{num}: score={score:.6f}, mean score={m1:.6f},"
          f" epochs={len(history.epoch)}, mean epochs={int(m2)}",
          f" time={hms_string(time_took)}")

# End Loop-----------------------------------------------------------------

# Print Final elapsed time
elapsed_time = time.time() - total_start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))


If your code is correct you should see the following output:

~~~text
STARTING 5 BOOTSTRAP CYCLES...
------------------------------------------------------------
Bootstrap Cycle: 1
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
mean_benchmark=[8.580744]
#1: score=8.580744, mean score=8.580744, epochs=238, mean epochs=238  time=0:00:40.52
Bootstrap Cycle: 2
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
mean_benchmark=[8.580744, 8.236672]
#2: score=8.236672, mean score=8.408709, epochs=51, mean epochs=144  time=0:00:08.87
Bootstrap Cycle: 3
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
mean_benchmark=[8.580744, 8.236672, 7.421387]
#3: score=7.421387, mean score=8.079601, epochs=35, mean epochs=108  time=0:00:07.08
Bootstrap Cycle: 4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
mean_benchmark=[8.580744, 8.236672, 7.421387, 8.3256]
#4: score=8.325600, mean score=8.141101, epochs=31, mean epochs=88  time=0:00:04.65
Bootstrap Cycle: 5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
mean_benchmark=[8.580744, 8.236672, 7.421387, 8.3256, 8.562627]
#5: score=8.562627, mean score=8.225406, epochs=19, mean epochs=74  time=0:00:02.98
Elapsed time: 0:01:04.18
~~~

You should notice that there are a **_variety_** of `epochs`, `mean epochs` and `times` reported in the output above. While your output will be different, it should also show some degree of variety.

This variety is an integral part of the `boostrapping` process which **_randomly_** splits the x and y data into _n_ different "chunks" where _n_=number of splits. As you should expect, in splits where EarlyStopping caused the training to terminate after a lower number of epochs, the `time` for that split will also be lower.

## Bootstrapping for Classification

While the code for Bootstrapping for Classification is similar the code above used for regression, we need to use different code for creating the boostrap object. As we saw in _K_-fold cross-validation, it is important, when the goal is classification, to **_balance the classes_** when generating the splits.

"Balancing" isn't necessary when performing regression. As long as the data is shuffled, any data sample should have roughtly the same distribution of high, low and medium values. But this is not necessarily true when it come to classification, especially if the number of items/subjects/patients in each class is significantly different. Even if the data is shuffled, there is some possibility that the random selection might inadvertly include too many samples from one class and not enough samples from a another class.

For example, imagine a dataset that contains the blood types of subjects who are `white`, `black` or `Asian`. And suppose that the number of `Asian` subjects is significantly smaller that the number of `white` subjects. If you randomly selected a subset of this dataset (what is called **Simple Random Sampling** in statistics), you might end up with a sample with very few or even no `Asian` subjects, and a disproportionate high number of `white` subjects. Training your neural network on this unbalanced subset sample would give you very bad results when tested later on a more general population sample.

In statistics, the way to avoid this problem is called **_Stratified Random Sampling_**. The population is divided into groups (strata), and members are randomly chosen from each group. This approach ensures really equal representation from every group (e.g., blood types of `white`, `black` or `Asian` subjects).

Our regression bootstrapping, in Example 1, just used the **ShuffleSplit()** class to perform the splits. But in buiding our classification neural network below, we will need to use instead, the **StratifiedShuffleSplit** class to perform the splits.  This class is similar to **StratifiedKFold** for cross-validation, as the classes are balanced so that the sampling does not affect proportions.  

### **Exercise 1 - Step 1: Create feature vector**

In the cell below, create a feature vector from the Body Performance dataset that can be used in a classification neural network. For the most part, the code for **Exercise 1 -Step 1** should be **_exactly_** the same as that used in Example 1 - Step 1, with the few exceptions noted below.

The main difference in preparing your feature vector will be the column you use to create your y-values. In Example 1, the y-values were `age`. Using 'age' for a regression neural network but not for a classification neural network.

For **Exercise 1 - Step 1** you will the y-values that are in the column `class`, that assigns a different performance level to each subject. In other words, you are building a neural network that can classify a subject's athletic `class`: `A`, `B`, `C` and `D`, based on their performance.  

You should use the same code that was shown in Example 1 - Step 1, but you will need to make the following changes:

1. When you create your x column list use this code:
~~~text
# Generate list of columns for x
x_columns = df.columns.drop('class')  # `class` is y-value
~~~
Since `class` is your y-value, you don't want it included with the dependent (x) variables.

2. Do **not** use this code to create dummies for the column `class`:
~~~text
# Generate dummies for class
df = pd.concat([df,pd.get_dummies(df['class'],prefix="class")],axis=1)
df.drop('class', axis=1, inplace=True)
~~~

Instead, use this code to One-Hot encode the column `class`:
~~~text
# One-Hot encode y-values for classification
dummies = pd.get_dummies(df['class']) # Classification
categories = dummies.columns
y = dummies.values
y = np.asarray(y).astype('float32')
~~~

3. At the end of the cell, print out the y-categories using this code chunk:
~~~text
# Print y categorical names
print(*categories)
~~~


In [ ]:
# Insert your code for Exercise 1 - Step 1 here



If your code is correct you should see the following output.
~~~text
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]
~~~

These are the four "classes" of performance that your neural network will predict based on performance.

### **Exercise 1 - Step 2: Bootstrapping for Classification**

In the cell below, write the code needed to create a classification neural network that uses bootstrapping. As with Step A above, your code should be **_exactly_** the same as that used in Example 1 - Step 2, except with the following exceptions:

Specific code differences:

1. Do **not** use this code to define the bootstrap:
~~~text
# Bootstrap
boot = ShuffleSplit(n_splits=SPLITS, test_size=0.1, random_state=42)
~~~

Instead use this code to define the bootstrap:
~~~text
# Bootstrap
boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1,
                                random_state=42)
~~~
Remember, you need to use the stratified version to make sure each class is equally represented.

2. Use this code to construct your neural network:
~~~text
def create_and_compile_model(input_shape, output_shape):
    model = Sequential()
    model.add(Input(shape=(input_shape,)))  # Input
    model.add(Dense(20, activation='relu'))  # Hidden 1
    model.add(Dense(10, activation='relu'))  # Hidden 2
    model.add(Dense(output_shape, activation='softmax'))  # Output layer for multi-class classification
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model
~~~

3. In the section called `# Measure this bootstrap's log loss` use this code section:
~~~text
    # Measure this bootstrap's log loss
    y_compare = np.argmax(y_test,axis=1) # For log loss calculation
    score = metrics.log_loss(y_compare, pred)
    mean_benchmark.append(score)
    m1 = statistics.mean(mean_benchmark)
    m2 = statistics.mean(epochs_needed)
    mdev = statistics.pstdev(mean_benchmark)
~~~

4. Use this code at the end of the loop:

~~~text
    # Record this iteration
    time_took = time.time() - start_time
    print(f"#{num}: score={score:.4f}, mean score={m1:.4f}," +\
          f"stdev={mdev:.4f}, epochs={num}, mean epochs={int(m2)}," +\
          f" time={hms_string(time_took)}")
~~~


We now run this data through a number of splits specified by the SPLITS variable. We track the average error through each of these splits.

In [ ]:
# Insert your code for Exercise 1 - Step 2 here




If your code is correct you should see something similar to the following output:

~~~text
STARTING 5 BOOTSTRAP CYCLES...
------------------------------------------------------------
Bootstrap Cycle: 1
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
#1: score=0.9386, mean score=0.9386,stdev=0.0000, epochs=1, mean epochs=90, time=0:00:18.56
Bootstrap Cycle: 2
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
#2: score=0.8420, mean score=0.8903,stdev=0.0483, epochs=2, mean epochs=147, time=0:00:35.08
Bootstrap Cycle: 3
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
#3: score=0.8087, mean score=0.8631,stdev=0.0551, epochs=3, mean epochs=105, time=0:00:05.11
Bootstrap Cycle: 4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
#4: score=0.7320, mean score=0.8303,stdev=0.0742, epochs=4, mean epochs=86, time=0:00:04.26
Bootstrap Cycle: 5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
#5: score=0.8184, mean score=0.8279,stdev=0.0665, epochs=5, mean epochs=72, time=0:00:03.04
Elapsed time: 0:01:06.15

~~~

## **Exercise/Example 2: Benchmarking**

Now that we've seen how to bootstrap with both classification and regression, we can start to try to **_optimize_** the hyperparameters for the **Body Performance** data.  For this example, we will encode for classification of the `class` column.  Evaluation will be in log loss.

### **Exercise 2: Create feature vector**

Here we are going to change things around a little bit. Instead of giving you an example followed by an exercise, here you will be given **Exercise 2** first followed by Example 2. The "catch" is that Example 2 won't run correctly unless your **Execise 2** runs correctly!

For Exercise 2, you are to create a feature vector from the Body Performance dataset that can be used in a classification neural network. The code for **Exercise 2 - Step 1** should be **_exactly_** the same as that used in **Exercise 1 - Step 1** above. As in **Exercise 1 - Step 1**, the y-values will be in the column `class`.

In [ ]:
# Insert your code for Exercise 2 here



If your code is correct you should see the following output:
~~~text
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]
~~~

### Example 2: Optimized Hyperparameters

We end this lesson with the following example in which some optimization has been performed on the code in the cell below. The code has the best settings that could be manually determined. Later in this course, we will see how we can use an automatic process to optimize the hyperparameters.

The architecture of this semi-optimized neural network has 3 hidden layers with 2 dropout layers, one after the first and one after the second hidden layer. Since the objective is classification, the bootstrap object is created using `StratifiedShuffleSplit()` to insure balanced classes.  

L2 regularizers are also used to reduce the effects of overfitting by adding a penalty to the synaptic weigths.

In [ ]:
# Example 2:

import pandas as pd
import os
import numpy as np
import time
import statistics
from sklearn import metrics
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Dropout, PReLU, Input
from keras.callbacks import EarlyStopping
from keras import regularizers

# Set variables
SPLITS = 5
EPOCHS = 1000

# Consistent tensor shapes
input_shape = x.shape[1]
output_shape = y.shape[1]  # Assuming y is one-hot encoded

def create_and_compile_model(input_shape, output_shape):
    model = Sequential()
    model.add(Input(shape=(input_shape,)))  # Input
    model.add(Dense(100, activation=PReLU(), kernel_regularizer=regularizers.l2(1e-4)))  # Hidden 1
    model.add(Dropout(0.5))  # Add dropout after Hidden 1
    model.add(Dense(100, activation=PReLU(), activity_regularizer=regularizers.l2(1e-4)))  # Hidden 2
    model.add(Dropout(0.5))  # Add dropout after Hidden 2
    model.add(Dense(100, activation=PReLU(), activity_regularizer=regularizers.l2(1e-4)))  # Hidden 3
    model.add(Dense(output_shape, activation='softmax'))  # Output layer
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

# Create the model outside the loop
model = create_and_compile_model(input_shape, output_shape)

# Record start time
total_start_time = time.time()

# Bootstrap
boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1, random_state=42)

# Track progress
mean_benchmark = []
epochs_needed = []
num = 0

print(f"STARTING {SPLITS} BOOTSTRAP CYCLES...")
print("-----------------------------------------------------------------")

# Start Loop......................................
for train, test in boot.split(x, df['class']):
    start_time = time.time()
    num += 1
    print(f"Bootstrap Cycle: {num}")

    # Split train and test
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]

    # Create Early Stopping monitor
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
                            patience=100, verbose=0, mode='auto', restore_best_weights=True)

    # Reset model weights before training
    model = create_and_compile_model(input_shape, output_shape)

    # Train on the bootstrap sample
    history = model.fit(x_train, y_train, validation_data=(x_test, y_test),
                        callbacks=[monitor], verbose=0, epochs=EPOCHS)

    # Record the number of epochs used
    epochs = len(history.epoch)
    epochs_needed.append(epochs)

    # Predict on the out-of-boot (validation)
    pred = model.predict(x_test)

    # Measure this bootstrap's log loss
    y_compare = np.argmax(y_test, axis=1)  # For log loss calculation
    score = metrics.log_loss(y_compare, pred)
    mean_benchmark.append(score)
    m1 = statistics.mean(mean_benchmark)
    m2 = statistics.mean(epochs_needed)
    mdev = statistics.pstdev(mean_benchmark)

    # Record this iteration
    time_took = time.time() - start_time
    print(f"#{num}: score={score:.4f}, mean score={m1:.4f},"
          f"stdev={mdev:.6f}, epochs={epochs},"
          f"mean epochs={int(m2)}, time={hms_string(time_took)}")

# End Loop ................................................

# Record end time
total_end_time = time.time()
print(f"Total Elapsed time = {hms_string(total_end_time - total_start_time)}")


If the code is correct you should see something similar to the following output:

~~~text
STARTING 5 BOOTSTRAP CYCLES...
-----------------------------------------------------------------
Bootstrap Cycle: 1
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
#1: score=0.8269, mean score=0.8269,stdev=0.000000, epochs=386,mean epochs=386, time=0:01:45.59
Bootstrap Cycle: 2
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
#2: score=0.8737, mean score=0.8503,stdev=0.023412, epochs=496,mean epochs=441, time=0:02:14.32
Bootstrap Cycle: 3
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
#3: score=0.8899, mean score=0.8635,stdev=0.026723, epochs=453,mean epochs=445, time=0:02:09.84
Bootstrap Cycle: 4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
#4: score=0.8340, mean score=0.8562,stdev=0.026443, epochs=653,mean epochs=497, time=0:02:55.29
Bootstrap Cycle: 5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
#5: score=0.9560, mean score=0.8761,stdev=0.046426, epochs=635,mean epochs=524, time=0:02:46.41
Total Elapsed time = 0:11:51.46
~~~

### **GPU ACCELERATION**

I re-ran the same code for Example 2 (above) but changed the `Runtime` to use the best GPU acceleration currently offered by GOOGLE Colab with a subscription. As you can see from the output below, on this particular piece of code, using GPU acceleration decreased the runtime by about 50%. So instead of waiting 12 minutes to run, the code ran in only 6 minutes.

~~~text
STARTING 5 BOOTSTRAP CYCLES...
-----------------------------------------------------------------
Bootstrap Cycle: 1
7/7 [==============================] - 0s 2ms/step
#1: score=0.7247, mean score=0.7247,stdev=0.000000, epochs=585,mean epochs=585, time=0:01:31.41
Bootstrap Cycle: 2
7/7 [==============================] - 0s 2ms/step
#2: score=0.8534, mean score=0.7890,stdev=0.064383, epochs=355,mean epochs=470, time=0:00:55.33
Bootstrap Cycle: 3
7/7 [==============================] - 0s 2ms/step
#3: score=0.8186, mean score=0.7989,stdev=0.054380, epochs=561,mean epochs=500, time=0:01:27.83
Bootstrap Cycle: 4
7/7 [==============================] - 0s 2ms/step
#4: score=0.8334, mean score=0.8075,stdev=0.049404, epochs=385,mean epochs=471, time=0:00:59.91
Bootstrap Cycle: 5
7/7 [==============================] - 0s 2ms/step
#5: score=0.9128, mean score=0.8286,stdev=0.061033, epochs=463,mean epochs=469, time=0:01:11.50
Total Elapsed time = 0:06:06.00
~~~

## **Lesson Turn-in**

When you have completed all of the code cells, and run them in sequential order (the last code cell should be number 9) use the **File --> Print.. --> Save to PDF** to generate a PDF of your JupyterLab notebook. Save your PDF as `Class_05_5.lastname.pdf` where _lastname_ is your last name, and upload the file to Canvas.